# Fine-Tuning LLaMA-2-7B with LoRA on Dolly-15K Dataset

## Purpose
This notebook implements **Parameter-Efficient Fine-Tuning (PEFT)** using **LoRA (Low-Rank Adaptation)** to fine-tune the **meta-llama/Llama-2-7b** model on the **databricks/databricks-dolly-15k** dataset. The goal is to transform the base model into a helpful conversational assistant that follows instructions effectively.

## Background
Instruction fine-tuning adapts a pretrained LLM to follow natural-language instructions instead of generic next-token prediction. This converts a raw pretrained model into a usable conversational assistant that can:
- Respond concisely to user instructions
- Follow specified formats consistently  
- Avoid generating unnecessary tokens
- Maintain high conversational quality

## Dataset: Dolly-15K
- **Source**: [databricks/databricks-dolly-15k](https://huggingface.co/datasets/databricks/databricks-dolly-15k)
- **Size**: 15,011 high-quality instruction-following examples
- **Format**: Each entry contains `instruction`, `context`, and `response` fields
- **Categories**: 7 different task categories (creative writing, information extraction, etc.)
- **Splits**: 80% train / 10% validation / 10% test

## Technical Approach
- **Base Model**: `meta-llama/Llama-2-7b-hf`
- **Fine-tuning Method**: LoRA (Low-Rank Adaptation)
- **Hardware**: Optimized for Colab with FSDP if VRAM is restricted
- **Training**: Track training and validation loss for convergence monitoring



## Expected Outcomes
- Upload fine-tuned weights to Colab for reuse and evaluation

## Workflow
1. **Environment Setup**: Install dependencies and mount Google Drive
2. **Data Preprocessing**: Load and format Dolly-15K dataset
3. **Model Configuration**: Set up LoRA parameters and training config
4. **Training**: Fine-tune with loss monitoring and checkpointing
5. **Model Saving**: Upload weights to Colab for later evaluation
6. **Documentation**: Record hyperparameters and training metrics

---
**Note**: This fine-tuning process is designed to demonstrate the effectiveness of LoRA for instruction-following capabilities while maintaining computational efficiency.


In [ ]:
!pip install -U transformers peft bitsandbytes accelerate trl datasets

In [ ]:
from datasets import load_dataset
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer, SFTConfig
import os
from peft import LoraConfig, get_peft_model

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)



print(f"✓ Model {model_id} loaded in 4-bit (QLoRA)")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(model)


In [ ]:
#LoRA Config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
drive_path = '/content/drive/MyDrive/LLaMA2-Dolly-Training/data'
os.makedirs(drive_path, exist_ok=True)
train_path = os.path.join(drive_path, 'train.parquet')
val_path = os.path.join(drive_path, 'val.parquet')
test_path = os.path.join(drive_path, 'test.parquet')


train_dataset_hf = load_dataset('parquet', data_files={'train': train_path})['train']
val_dataset_hf = load_dataset('parquet', data_files={'validation': val_path})['validation']
test_dataset_hf = load_dataset('parquet', data_files={'test': test_path})['test']


In [ ]:
# TRAINING CONFIGS

output_dir = "/content/drive/MyDrive/LLaMA2-Dolly-Training/results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
learning_rate = 2e-4
max_grad_norm = 0.3
num_train_epochs = 1
warmup_ratio = 0.03
lr_scheduler_type = "constant"
save_strategy = "steps"
save_steps = 100
logging_strategy = "steps"
logging_steps = 10
eval_steps = 50
max_seq_length = 1024

# --- Create SFTConfig ---
sft_config = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_strategy=save_strategy,
    save_steps=save_steps,
    logging_strategy=logging_strategy,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_length=max_seq_length,
    num_train_epochs=num_train_epochs,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    do_eval=True,                # Enable evaluation
    eval_steps=eval_steps,       # Evaluation frequency
    save_total_limit=2,
    report_to="none",
    dataset_text_field="text",
    packing=False,
)

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset_hf,
        eval_dataset=val_dataset_hf,
        peft_config=peft_config,
        processing_class=tokenizer,
        args=sft_config,
  )

In [ ]:
print("\nStarting QLoRA fine-tuning...")
start_time = time.time()

trainer.train()

end_time = time.time()
training_duration_minutes = (end_time - start_time) / 60
print(f"Training finished in: {training_duration_minutes:.2f} minutes")

final_adapter_path = os.path.join(output_dir, "final_lora_adapter")
print(f"\nSaving final LoRA adapter state to: {final_adapter_path}")
trainer.model.save_pretrained(final_adapter_path)
tokenizer.save_pretrained(final_adapter_path)
print("Final adapter and tokenizer saved.")